# Clonamos el repositorio para obtener los dataSet

In [ ]:
!git clone https://github.com/joanby/machinelearning-az.git

# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [ ]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Instalar dependendias

In [ ]:
#!pip install sklearn

# Instalar Theano

In [8]:
# pip install --upgrade --no-deps git+https://github.com/Theano/Theano.git  
# librería de desarrolladores para llevar a cabo calculos matemáticos complejos (la instalamos en Anaconda prompt)

# Instalar Tensorflow y Keras

In [2]:
#!pip install keras # Google, se basa en TensorFlow, simplificando aún más la preparación de NN para que sea menos técnica su preparación
#!pip install tensorflow # Google (permite crear NN sencillas)
# Tambien las librerías nos permiten la opción de utilizar la GPU en lugar de la CPU para ejecutar los modelos (más potente por poseer más núcleos)

In [ ]:
# Para instalar la última versión de keras (y con ello incluida tensorflow), ponemos en Anaconda prompt:
# -c conda-forge keras

# Redes Neuronales Artificales

# Cómo importar las librerías


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values # no tomamos el número de filas, el id ni el apellido
y = dataset.iloc[:, 13].values # si abandonó o no el clinete
# Conjunto de datos bancario, 10000 clientes, el banco detecta una tasa de abandono elevada, y nuestra tarea es buscar los motivos (credit score como índice recopilado a partir de solvencia de cada cliente), siendo exited=1 cuando la persona ya no está en el banco y exited=0 es que sigue
# Este criterio de clasificación problematica de una empresa, puede aplicarse a cualquier caso (transacciones fraudulentas, etc). Inlcuso podríamos extraer posibilidades de que cada cliente se vaya
# Optamos por las NN porque tenemos muchas variables independientes, y mucha información

# Parte 1 - Pre procesado de datos

# Codificar datos categóricos

In [4]:
# Con NN también debemos transformar las variables categóricas a ficticias (también podríamos codificar y si hiciera falta)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) # codifico la variable Geografía (Francia, España o Alemania)
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) # codifico la variable Género (Masculino, Femenino)
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
) 
X = onehotencoder.fit_transform(X) # transformo en ficticias (Dummy) las varaibles Geografía y Género, evitando caer en la trampa de las ficticias eliminando 1 de cada tipo para evitar Multicolinealidad
X = X[:, 1:] # elimino una variable ficticia de Geografía (para Género no me hace falta)

# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Escalado de variables

In [6]:
# Es obligatorio en NN hacer reescalado (estandarizado) de los datos, para que la red no priorice variables con valores mayores sobre otras
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test) # aplico la transformación ya ajustada en train

# Parte 2 - Construir la RNA

# Importar Keras y librerías adicionales


In [7]:
import keras
from keras.models import Sequential # red neuronal secuencial con pesos aleatorios inicializados
from keras.layers import Dense # conexiones entre capas 

# Inicializar la RNA

In [8]:
classifier = Sequential() # Creamos la red neuronal para luego agregarle capas secuenciales

# Añadir la capa de entrada y primera capa oculta


In [9]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform", # Dense me añade la conexión entre capas, units = número de nodos/neuronas en la capa, kernel_initializer = con que distribución se generan los pesos al inicio del algoritmo (uniform sería aleatorio y cercano a 0, pero podríamos hacerlo con una constante, etc), input_dim = dimensión de los valores de entrada (número de variables independientes), podemos usar input_shape en lugar de esta, a la que le tendríamos que aclarar primero el batch_size primero (si no lo sabemos dejar espacio en blanco, indicando que es variable)
                     activation = "relu", input_dim = 11)) # primera capa oculta conformada por 6 neuronas de salida (regla general para neuronas por capas ocultas = media de la capa de entrada de la capa específica + capa de salida de la red, pero experimentar), la cual va a recibir 11 valores de entrada (11 variables explicativas tengo para cada dato del dataset)
# Usamos f de activación ReLu para que cada neurona se active si la información o el análisis que posee es relevante/útil para predecir Y (interesante para capas intermedias). Pero podríamos usar cualquier otra

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Añadir la segunda capa oculta

In [10]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu")) # segunda capa oculta de 6 neuronas de salida (podríamos poner de 3 neuronas, ya que es el promedio entre lo que recibe, osea su capa de entrada + la capa de salida de la red), ya sabe lo que espera de la capa anterior, por lo que no hace falta poner input_dim=6
# Usamos ReLu para que cada neurona se active si la información o el análisis que posee es relevante/útil para predecir Y (interesante para capas intermedias). Podemos dejar uniform, ya que solo es importante para la primera capa

# Añadir la capa de salida

In [11]:
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid")) # capa de salida de 1 neurona (clasificación binaria)
# Usamos Sigmoid para ver la función de probabilidad de cada cliente de dejar el banco o no (P(y=1) entre 0 y 1), estableciendo un punto de corte a partir del cual la función de activación asigne y prediga
# Podriamos usar lineal para que directamente me lo prediga como 1 o 0
# Si se quiere clasificar en 3 categorías, por ejemplo, tendríamos que asignar a la capa de salida 3 nodos/neuronas y la función de activación deberia ser Softmax, para que todas las probabilidades sigmoidales de las categorías sumen 1 (o podríamos usar ReLu para que se active según cada predicción)

# Compilar la RNA 

In [12]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"]) # optimizer = algoritmo que se usa para buscar y encontrar el conjunto óptimo de pesos de la NN (Gradiente Descendiente, GD Estocástico, Adam(recomendado, es estocástico al igual que GDE, pero está más optimizado)), loss = función que debemos minimizar su valor, representado por la diferencia entre el valor predicho y el real (OLS(min cuad ordinarios), entropía cruzada (diferencias entre categorias, binaria o múltiple)), metrics = metrica para evaluar el modelo, que intentará mejorar la red a medida que itere minimizando el error (solemos usar precisión (calculado en base a matriz confusión), pero podemos poner corchetes y agregar varias) 
# preparamos la red neuronal para que funcione (enganchar nodos, asignar pesos al inicio, etc)

# Ajustamos la RNA al Conjunto de Entrenamiento

In [13]:
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100) # batch learning (toma los datos en mini-bloques para actualizar los pesos una vez ejecutado) o reinforcement learning si queremos que actualice luego de recibir cada dato (más costoso computacionalmente y menos óptimo). Que no sea el batch tan grande así es capaz de captar mínimos globales mediante grandes saltos en la curva de costes al obligar a la red a actualizar pesos cada pocos datos
# Le decimos que ajuste la red a los datos tomando de a bloques de 10 filas/datos para que luego de ejecutarlos/procesarlos actualice los pesos según el error acumulado hasta el momento, y que ejecute 100 épocas (100 recorridos completos al set de train yendo batch por batch en cada uno)
# Ojo con entrenar el modelo en épocas de más, ya que podemos sobreajustar los datos para que se acoplen a los de entrenamiento, perdiendo capacidad predictiva (Overffiting)

# Podemos probar cambiando los parámetros, e incluso podemos quedarnos con los pesos de la mejor época con checkpoints (en la que más accuracy presente, al mismo tiempo de que menos valor en la función de perdida, pero ojo con que no genere Overffiting)
# La precisión de predicción de la NN con respecto al set de datos de entrenamiento gira en torno al 86-87%

Epoch 1/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7933 - loss: 0.5907
Epoch 2/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8425 - loss: 0.3909
Epoch 3/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8527 - loss: 0.3622
Epoch 4/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8555 - loss: 0.3511
Epoch 5/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8579 - loss: 0.3477
Epoch 6/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8584 - loss: 0.3405
Epoch 7/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8580 - loss: 0.3417
Epoch 8/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8566 - loss: 0.3433
Epoch 9/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8608 - loss: 0.3484
Epoch 10/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8609 - loss: 0.3367
Epoch 11/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8688 - loss: 0.3339
Epoch 12/100
800/800 ━━━━━━━━━━━━━━━━━━━━

# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing

In [16]:
y_pred  = classifier.predict(X_test) # nos predice las probabilidades (salida sigmoidal) de que cada cliente abandone el banco. Puede ser interesante presentar los datos en este formato
y_pred = (y_pred>0.5) # nos transforma la predicción en True(1) o False(0) según (supere) o (menor/igual) que 0.5, respectivamente. Pero el umbral de decisión es a criterio del decisor, pudiendo querer obtener los clientes que la red identifique como más seguros de abandonar, o menos (en terreno médico se suele elegir un umbral mucho menor, para ser precavidos).
# Todos los que esten > 0.5 true y <= 0.5 False

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Elaborar una matriz de confusión

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm) 
print((1511 + 213)/2000) # Accuracy de la NN con respecto a los datos de test (no se produce Overffiting, ya que es similar la precisión sobre el conjunto de train y sobre el de test. Si fuera mucho mayor en train que en test si)
# Forma de evaluar las predicciones van a ser las mismas que en los algoritmos de ML

[[1511   84]
 [ 192  213]]
0.862
